# **CLASS_1**

**What good can ai in robotic fabrication do?**

1. Make intelligent desicions
2. adapt to changes
3. optimise fabrication tasks

**Do we nned ai in robotic fabrication?**

it is usefull for the following applications:
1. computer vision for real-time feedback
2. Reinforcement learning for adaptive control
3. ai driven toolpath optimization
4. generative ai for design & fabrication
5. HUman-Robot collaboration

## Examples of uses of ai in robotic fabrication:

**How do you prototype?**

What-why-how needs to be answered<br>
concepts of operationsdefinition<br>
design<br>
fabrication<br>

**questions to answer:**

where do we get the data?<br>
how do we test? how do we simulate?<br>

it is suggested that we work to get the fabrication data.

# **CLASS_2**

reinforcement learning:

1. it is all about trial and error
2. instead of being told what to do, an agent learns by interacting with the environment
3. inspired by desicion making and robotics.


wokflow:


use cases:

HVAC Contorl
urban planning
autonomous construction robots
floor plan optimization

## Reinfeorcement Learning exercises

### 1. paddle game

In [ ]:
import gymnasium as gym
import numpy as np
import pygame
from gymnasium import spaces

class CatchGameEnv(gym.Env):
    def __init__(self, width=400, height=400):
        super().__init__()
        self.width, self.height = width, height
        self.paddle_width = 80
        self.paddle_x = (width - self.paddle_width) // 2
        self.block_x = np.random.randint(0, width - self.paddle_width)
        self.block_y = 0
        self.block_speed = 5
        self.done = False
        
        # Define action & observation spaces
        self.action_space = spaces.Discrete(3)  # Move left, stay, move right
        self.observation_space = spaces.Box(low=0, high=width, shape=(2,), dtype=np.int32)

        # Initialize pygame
        self.screen = pygame.display.set_mode((width, height))
        pygame.display.set_caption("Catch the Falling Block")
        self.clock = pygame.time.Clock()

    def reset(self, seed=None, options=None):
        self.paddle_x = (self.width - self.paddle_width) // 2
        self.block_x = np.random.randint(0, self.width - self.paddle_width)
        self.block_y = 0
        self.done = False
        return np.array([self.paddle_x, self.block_x]), {}

    def step(self, action):
        if action == 0:  # Move left
            self.paddle_x = max(0, self.paddle_x - 20)
        if action == 2:  # Move right
            self.paddle_x = min(self.width - self.paddle_width, self.paddle_x + 20)

        self.block_y += self.block_speed  # Block falls down

        # Check if block reaches bottom
        if self.block_y >= self.height - 20:
            reward = 10 if abs(self.paddle_x - self.block_x) < 40 else -10
            self.done = True
        else:
            reward = 0
        
        return np.array([self.paddle_x, self.block_x]), reward, self.done, False, {}

    def render(self):
        self.screen.fill((0, 0, 0))
        pygame.draw.rect(self.screen, (0, 255, 0), (self.paddle_x, self.height - 20, self.paddle_width, 10))
        pygame.draw.rect(self.screen, (255, 0, 0), (self.block_x, self.block_y, 20, 20))
        pygame.display.flip()
        self.clock.tick(30)

    def close(self):
        pygame.quit()

env = CatchGameEnv()

q_table = np.zeros((400, 400, 3))  # Q-values for (paddle_x, block_x, action)
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.1  # Exploration rate

for episode in range(1000):
    state, _ = env.reset()
    done = False
    
    while not done:
        paddle_x, block_x = state
        if np.random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()  # Explore
        else:
            action = np.argmax(q_table[paddle_x, block_x])  # Exploit

        next_state, reward, done, _, _ = env.step(action)
        next_paddle_x, next_block_x = next_state

        # Q-learning update
        q_table[paddle_x, block_x, action] = (1 - alpha) * q_table[paddle_x, block_x, action] + \
            alpha * (reward + gamma * np.max(q_table[next_paddle_x, next_block_x]))

print("Training complete!")

state, _ = env.reset()
done = False

while not done:
    paddle_x, block_x = state
    action = np.argmax(q_table[paddle_x, block_x])  # Use trained policy
    state, reward, done, _, _ = env.step(action)
    env.render()

env.close()

### Lunar lander

In [ ]:
import gymnasium as gym
import numpy as np
import random
import time

env = gym.make("LunarLander-v3", render_mode="human")

# Discretization settings
num_bins = 10  # Fewer bins = faster learning, but less precision
state_bins = [np.linspace(low, high, num_bins) for low, high in zip(env.observation_space.low, env.observation_space.high)]

# Q-table initialization
q_table = np.zeros([num_bins] * len(env.observation_space.low) + [env.action_space.n])

def discretize_state(state):
    """Convert continuous state into discrete bins."""
    return tuple(np.digitize(state[i], state_bins[i]) - 1 for i in range(len(state)))

# Hyperparameters
alpha = 0.1  # Learning rate
gamma = 0.99  # Discount factor
epsilon = 1.0  # Initial exploration rate
epsilon_decay = 0.995  # Decay factor
min_epsilon = 0.01
num_episodes = 50000  # Increase for better learning

# Training loop
for episode in range(num_episodes):
    state, _ = env.reset()
    state = discretize_state(state)
    done = False
    total_reward = 0

    while not done:
        # Choose action (exploration-exploitation tradeoff)
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()  # Explore
        else:
            action = np.argmax(q_table[state])  # Exploit best known action

        next_state, reward, done, _, _ = env.step(action)
        next_state = discretize_state(next_state)

        # Q-learning update
        best_next_action = np.argmax(q_table[next_state])
        q_table[state][action] = (1 - alpha) * q_table[state][action] + alpha * (reward + gamma * q_table[next_state][best_next_action])

        state = next_state
        total_reward += reward

    # Decay exploration rate
    epsilon = max(min_epsilon, epsilon * epsilon_decay)

    if episode % 1000 == 0:
        print(f"Episode {episode}, Total Reward: {total_reward}, Epsilon: {epsilon:.4f}")

print("Training complete!")

state, _ = env.reset()
state = discretize_state(state)
done = False

while not done:
    action = np.argmax(q_table[state])  # Select best action
    state, reward, done, _, _ = env.step(action)
    state = discretize_state(state)
    env.render()
    time.sleep(0.01)  # Slow down rendering for visibility

env.close()

### Floor plan

In [ ]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces
import random

class FloorPlanEnv(gym.Env):
    def __init__(self):
        super(FloorPlanEnv, self).__init__()
        self.grid_size = 5
        self.observation_space = spaces.Box(low=0, high=1, shape=(self.grid_size, self.grid_size), dtype=np.int8)
        self.action_space = spaces.Discrete(4)  # Move room: up, down, left, right
        self.room_position = [2, 2]  # Start in the center

    def reset(self):
        self.room_position = [2, 2]
        return self._get_obs(), {}

    def step(self, action):
        if action == 0: self.room_position[0] = max(0, self.room_position[0] - 1)  # Up
        if action == 1: self.room_position[0] = min(self.grid_size - 1, self.room_position[0] + 1)  # Down
        if action == 2: self.room_position[1] = max(0, self.room_position[1] - 1)  # Left
        if action == 3: self.room_position[1] = min(self.grid_size - 1, self.room_position[1] + 1)  # Right
        
        reward = -1  # Penalize unnecessary moves
        if self.room_position == [4, 4]:  # Goal: reach optimal placement
            reward = 10  
            done = True
        else:
            done = False
        
        return self._get_obs(), reward, done, False, {}

    def _get_obs(self):
        obs = np.zeros((self.grid_size, self.grid_size), dtype=np.int8)
        obs[self.room_position[0], self.room_position[1]] = 1
        return obs

env = FloorPlanEnv()

q_table = np.zeros((5, 5, 4))  # Q-values for each (x, y, action)
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.1  # Exploration rate

for episode in range(500):
    state, _ = env.reset()
    done = False
    
    while not done:
        x, y = env.room_position
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()  # Explore
        else:
            action = np.argmax(q_table[x, y])  # Exploit
        
        next_state, reward, done, _, _ = env.step(action)
        next_x, next_y = env.room_position

        # Q-learning update
        q_table[x, y, action] = (1 - alpha) * q_table[x, y, action] + alpha * (reward + gamma * np.max(q_table[next_x, next_y]))

print("Training complete!")

state, _ = env.reset()
done = False

while not done:
    x, y = env.room_position
    action = np.argmax(q_table[x, y])
    next_state, reward, done, _, _ = env.step(action)
    print(f"Room moved to: {env.room_position}, Reward: {reward}")

# **CLASS_3**

# **PROJECT**

## 1. BRAINSTORM AND CHOOSING THE TOPIC

1. trasnforming a physiscal object into a dynamic 3d geometry through the use of sensing tools.

2. using ml approaches to make autonomous drones that allows them to detect and avoid obstacles.

3. use ai to control the coordination of robots: <br>
    3.1 use swarms to move objects. <br>
    3.2 use swarm to get from point a to point b using coordinaton between the robot caabilities. <br>
    3.3 use swarms to get robots to coordinate and build something architectural. <br>

4. translating flat drawings into drawings in not planar surfaces.

**INPUTS**

drawing image

**AI PROCESSING**

**ROBOTIC ACTION**

## Boat thingy taskies

aside from the entrance fee and the tickets, everything else is covered.

there will be a roadmap at some point 

we about to get informed about software tasks

1 team focuses on identification of the elements 

the other team focuses 



navigation

path planning 

collision 

localization.